In [1]:
%run ./Utils.ipynb

In [2]:
# Load dataset function

def load_dataset():
    train_X = np.load(f"train_{COLLECTION_FILE_NAME}_x.npy")
    train_Y = np.load(f"train_{COLLECTION_FILE_NAME}_y.npy")
    test_X = np.load(f"test_{COLLECTION_FILE_NAME}_x.npy")
    test_Y = np.load(f"test_{COLLECTION_FILE_NAME}_y.npy")
    return train_X, train_Y, test_X, test_Y
    
X_train, Y_train, X_test, Y_test = load_dataset() 
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(320, 10304) (320,) (80, 10304) (80,)


In [3]:
# Extra shuffle for samples
def shuffle_samples(x, y):
    data = np.concatenate((x, y.reshape(-1,1)), axis=1)
    np.random.shuffle(data)
    return data[:,:-1] , data[:,-1]

X_train, Y_train = shuffle_samples(X_train, Y_train)
print(X_train.shape, Y_train.shape)

(320, 10304) (320,)


In [4]:
# Default scaling
def minmaxscaler(samples):
    return samples/255.

# Scale images A
def scale_by_mean(samples):
    return samples - np.mean(samples)

# Scale images B
def scale_by_feature(samples):
    return (samples - np.mean(samples, axis=0))/np.std(samples, axis=0)

X_train = minmaxscaler(X_train)
X_test  = minmaxscaler(X_test)

In [5]:
# Function for encode categorical labels

from sklearn.preprocessing import OneHotEncoder

def encode_labels(df_train, df_test):
    df = np.concatenate((df_train, df_test))
    enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
    df = enc.fit_transform((df).reshape((-1, 1)))
    return df[:-80], df[320:]

Y_train_encoded, Y_test_encoded = encode_labels(Y_train, Y_test)
print(Y_train_encoded.shape, Y_test_encoded.shape)

(320, 40) (80, 40)


In [6]:
# PCA REALIZATION

from sklearn.decomposition import PCA

def vector_by_pca(train, test):
    pca = PCA(.97)
    pca.fit(train)
    return pca.transform(train), pca.transform(test)

X_train_PCA, X_test_PCA = vector_by_pca(X_train, X_test)
print(X_train_PCA.shape, X_test_PCA.shape)

(320, 202) (80, 202)


In [7]:
# Implementation of NN with helper functions

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def initialize_with_zeros(dim):
    w = np.zeros((dim, 40))
    b = 0

    assert(w.shape == (dim, 40))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

def count_loss(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)
    cost = -np.mean(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    return cost, m, A

def propagate(w, b, X, Y):
    cost, m, A = count_loss(w, b, X, Y)

    dw = np.dot(X, (A - Y).T) / m
    db = np.mean(A - Y)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, X_test, Y_test, print_cost = False):
    train_costs = []

    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        valid_loss, _, _ = count_loss(w, b, X_test, Y_test)

        
        if i % 10 == 0:
            train_costs.append(cost)

        if print_cost and i % 10 == 0:
            print ("Train cost after iteration %i: %f" %(i, cost))
            print ("Test cost after iteration %i: %f" %(i, valid_loss))
            
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, train_costs


def predict(w, b, X):
    m = X.shape[1]
    
    w = w.reshape(X.shape[0], 40)
    
    A = sigmoid(np.dot(w.T, X) + b)
    Y_prediction = []
    for i in range(A.shape[1]):
        max_index = np.argmax(A[:, i])
        zeros = np.zeros(A.shape[0])
        zeros[max_index] = 1. if A[max_index][i] >= 0.5 else 0. 
        Y_prediction.append(zeros)
    
    Y_prediction = np.asarray(Y_prediction).T
    
    assert(Y_prediction.shape == (40, m))
    
    return Y_prediction

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    w, b = initialize_with_zeros(X_train.shape[0])

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, X_test, Y_test, print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    print("train accuracy: {} %".format(100 * np.mean(np.argmax(Y_prediction_train, axis=0) == np.argmax(Y_train, axis=0))))
    print("test accuracy: {} %".format(100 * np.mean(np.argmax(Y_prediction_test, axis=0) ==  np.argmax(Y_test, axis=0))))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [8]:
d = model(X_train_PCA.T, Y_train_encoded.T, X_test_PCA.T, Y_test_encoded.T, num_iterations = 100, learning_rate = 0.005,print_cost= True)

Train cost after iteration 0: 0.693147
Test cost after iteration 0: 0.691620
Train cost after iteration 10: 0.678299
Test cost after iteration 10: 0.677491
Train cost after iteration 20: 0.664972
Test cost after iteration 20: 0.664699
Train cost after iteration 30: 0.652515
Test cost after iteration 30: 0.652695
Train cost after iteration 40: 0.640654
Test cost after iteration 40: 0.641244
Train cost after iteration 50: 0.629261
Test cost after iteration 50: 0.630230
Train cost after iteration 60: 0.618265
Test cost after iteration 60: 0.619589
Train cost after iteration 70: 0.607624
Test cost after iteration 70: 0.609282
Train cost after iteration 80: 0.597307
Test cost after iteration 80: 0.599282
Train cost after iteration 90: 0.587291
Test cost after iteration 90: 0.589566
train accuracy: 98.75 %
test accuracy: 90.0 %
